# Восстановление зависимости нейронной сетью

Задача восстановления некоторой зависимости - регрессия

Выборка делится обычно на 3 части:
- train - обучающая, для подбора параметров нейросети
- valid - для подбора гипер параметров обучения
- test - для подтверждения результатов

Пусть есть истинная зависимость $y = f(x)$ (например, скорости ветра от давления)

- есть наблюдения $y_i = f(x_i) + \epsilon_i$, где 
  - $y_i$ - наблюдаемая объясняемая (зависимая) переменная
  - $x_i$ - наблюдаемый параметр
  - $\epsilon_i$ - ошибка наблюдения, которая может быть любой природы:
    - влияние других неизвестных параметров
    - некорректные наблюдения/измерения
    - другие случайные и систематические ошибки

## Компоненты нейронной сети

- архитектура нейронной сети
- функция потерь
- метод оптимизации
- метрики (итогового качества решения задачи)

### Нейронная сеть

Пусть архитектура будет такая:
- вход 1
- выходной нейрон - линейный, т.е. выдает линейную комбинацию приходящих сигналов
- между ними "много" сигмоидных нейронов с 1 выходом и выходом

Нужно подобрать параметры выходного слоя (коэффициенты линейной комбинации) и скрытого слоя таким образом, чтобы выходной сигнал максимально близко аппроксимировал исходные данные $x_i, y_i$

<img src="./img/approx.png" width="700">

Т.е. целевая зависимость будет искаться в виде линейной комбинации отмасштабированных вдоль оси $x$ ($w_i x_i$) и сдвинутых на смещение ($b_i$) сигмоидных функций $\sigma(w_i x_i + b_i)$.
- первый нейрон должен хорошо приближать первую часть данных, второй - дальше, и т.д.

Т.о. можно построить сколь угодно точную аппроксимацию любой непрерывной функции: "универсальный аппроксиматор"

- **Теорема Цыбенко** (искусственная нейронная сеть прямой связи, в которых связи не образуют циклов, с одним скрытым слоем может аппроксимировать любую непрерывную функцию многих переменных с любой точностью)
- **Теорема Колмогорова — Арнольда** (каждая многомерная непрерывная функция может быть представлена в виде суперпозиции непрерывных функций одной переменной) 

Более того, при помощи линейной комбинации сигмоид можно с любой точностью приблизить любую ограниченную функцию с не более, чем счетным числом разрывов.

\*\* Существует масса других способов решать задачу регрессии. Например, это можно делать при помощи полиномов степени $N: P_N (x) = \sum_{i = 0}^N a_i x^i$ и рядов Фурье (в частности, систем тригонометрических функций).

**Разделяющая поверхность в задаче регрессии** - 

- т.е. когда у нас активационная функция сигмоида - это область значений, где функция активации принимает значение 0,5 (порог классификации)

Например, разделяющая поверхность для нейрона $y_1 = \sigma(300 \cdot x + 100)$ (почти ступенчатая функция), будет значительно левее по оси $x$, чем для нейрона $y_2 = \sigma(2 \cdot x - 1)$.

### Функция потерь

Частый вариант - средний квадрат ошибки: $MSE = \frac{1}{N} \sum (\hat y_i - y_i)^2$

### Алгоритм настройки нейронной сети

$W_0$ - исходное состояние сети (веса и смещения $b = w_0$)


$\nabla f = (\frac{\partial f}{\partial w_0}, ... \frac{\partial f}{\partial w_n})^T$ - вектор-градиент функции потерь

$W_{k+1} = W_k - \alpha \nabla f(W_{k-1})$ - очередной шаг градиентного спуска, уточненное состояние сети, уменьшающее функцию потерь при определенных условиях

Чтобы уменьшить функцию потерь мы должны знать насколько она изменяется при изменении любого параметра сети, т.е. мы должны знать все ее частные производные по всем параметрам.
- именно поэтому реальные целевые функции задачи (точность классификации и т.п. не могут использоваться в качестве функций потерь)
- нужны такие функции потерь (и такие функции активации). которые имеют производные, и чей минимум будет там же, где экстремум целевой функции задачи (метрики)
  - дифференцируемые в большинстве точек
    - итоговые целевые функции (метрики) не подходят (точность и т.п.)
  - не равны 0 в большинстве точек
    - пороговая функция активации не подходит

Например:

$$\frac{\partial MSE}{\partial \hat y_i} = \frac{2}{N} (\hat y_i - y_i)$$

#### Правило цепочки, производная сложной функции

Чтобы посчитать производные по всем параметрам всех слоев сети нужно брать производные сложных функций, которые образуют **граф вычисления**, который определяется топологие сети. 

А в основе просто:

$$\frac{\partial f(g(x), e(x))}{\partial x} = \frac{\partial f}{\partial g} \frac{\partial g}{\partial x} + \frac{\partial f}{\partial e} \frac{\partial e}{\partial x}$$

Еще раз - функция потерь сети это комбинация функций потерь и функций активации всех ее узлов, которые образуют граф вычисления. Обратное распространение ошибки - расчет производной функции потерь сети по всем, включая скрытые, параметрам, чтобы в ходе градиентного спуска изменять их все в направлении уменьшения функции потерь.

# Теоретические задачи: Графы вычислений и BackProp

Запишите математическое выражение, которое задает этот граф

<img src="./img/task_2_4_graph_01.png" width="400">

y = c1 * (x1 + b1) * sigma(x2 + b2) + sigma(x1 + b1) * c2 * tanh(x2 + b2)

Чему будет равна производная $y$ по параметру $с_2$ в терминах частных производных промежуточных результатов

d(y, c2) = d(y, z9) * d(z9, z6) * d(z6, c2)

Чему будет равна производная $y$ по параметру $b_1$

d(y, b1) = d(y, z8) * d(z8, z7) * d(z7, z1) * d(z1, b1) + d(y, z9) * d(z9, z3) * d(z3, z1) * d(z1, b1)

Чему будет равна производная $y$ по параметру $b_2$

d(y, b2) = d(y, z8) * d(z8, z7) * d(z7, z4) * d(z4,z2) * d(z2, b2) + d(y, z9) * d(z9, z6) * d(z6, z5) * d(z5, z2) *d(z2, b2)

Чему будет равна производная $\tilde{y}$ по $с_2$, выраженная через входные данные и параметры

d(y, c2) = tanh(x2 + b2) * sigma(x1 + b1)

Чему будет равна производная $\tilde{y}$ по $b_1$

d(y, b1) = c1 * sigma(x2 + b2) + c2 * tanh(x2 + b2) * sigma(x1 + b1) * (1 - sigma(x1 + b1))

Чему будет равна производная $\tilde{y}$ по $b_2$

d(y, b2) = c1 * (x1 + b1) * sigma(x2 + b2) * (1 - sigma(x2 + b2)) + sigma(x1 + b1) * c2 * (1 - tanh(x2 + b2)**2)

**Рассмотрим линейное преобразование:**
$$\vec{y} = W \vec{x} + \vec{b}$$
 
Допустим, что мы знаем производную 
$$\frac{\partial L}{\partial \vec{y}}$$

Обозначим ее $\vec{l}$.

Найдите производную
$$\frac{ \partial{L}} {\partial \vec{b}} = \frac{\partial L}{\partial \vec{y}} \frac{ \partial{y}} {\partial \vec{b}} = l$$

Найдите производную
$$\frac{ \partial{L}} {\partial \vec{x}} = \frac{\partial L}{\partial \vec{y}} \frac{ \partial{y}} {\partial \vec{x}} = W^T \cdot l$$

Найдите производную
$$\frac{ \partial{L}} {\partial W} = \frac{\partial L}{\partial \vec{y}} \frac{ \partial{y}} {\partial \vec{W}} = l \cdot x^T$$

# Теоретические задачи: Восстановление зависимостей

1. **Мы рассматривали задачу** аппроксимации наблюдаемой одномерной зависимости $\{ x_i, y_i \}_{i=1}^N$ при помощи набора сигмоидных функций. $i$ здесь — номер наблюдаемого обьекта, $x_i$ — координата замера, $y_i$ — замеренное значение.

Часто для решения этой задачи используется система степеней $\tilde{y}_i = \sum_{j=0}^M a_j x^j_i$, где $a_j$ — коэффициенты при степенях $x_i^j$, $\tilde{y}_i$ — аппроксимированное значение функции. Это выражение можно записать в матричном виде:

$\tilde{y} = X a$, где

$X = \begin{bmatrix} x_0^0 & x_0^1 & x_0^2 & \dots & x_0^M \\ x_1^0 & x_1^1 & x_1^2 & \dots & x_1^M \\ \vdots & \vdots & \vdots & \ddots & \vdots \\ x_N^0 & x_N^1 & x_N^2 & \dots & x_N^M \end{bmatrix} \\
a = \begin{bmatrix} a_0 & a_1 & a_2 & \dots & a_M \end{bmatrix}^T \\
\tilde{y} = \begin{bmatrix} \tilde{y}_0 & \tilde{y}_1 & \tilde{y}_2 & \dots & \tilde{y}_N \end{bmatrix}^T$ 

Если мы хотим точно решить задачу, то мы должны найти решение системы линейных уравнений: $Xa = y$.

Однако зачастую вместо непосредственного решения этого уравнения, которое сводится к вычислению обратной матрицы $X^{-1}$, решают задачу оптимизации (минимизации $L_2$-нормы):

$\| Xa - y \|_2^2 \rightarrow \min_{a}$, где $\| z \|_2$ — длина вектора $z$.

Докажите, что решение этой задачи оптимизации является и решением системы линейных уравнений $Xa = y$ в том случае, если эта система разрешима, вычислив производную от выражения $L(a) = \|Xa - y\|_2^2$ по всем параметрам $a$ и приравняв ее к нулю.

**Док-во**:

$$\| Xa - y \|_2^2 \rightarrow \min_{a} \Leftrightarrow \frac{\partial L}{\partial a} = X^T \cdot 2 (Xa - y) = 0 \Leftrightarrow Xa - y = 0 \Leftrightarrow Xa = y \Leftrightarrow a = X^{-1}y$$

- т.е. если линейная система уравнений разрешима, то ее можно решать при помощи минимизации нормы разности левой и правой частей системы. Это эквивалентные задачи.

2. **При какой степени** полинома $M$, полином $\sum_{j=0}^M a_j x_i^j$ может точно пройти по всем точкам $\{ x_i, y_i \}_{i=0}^N$?

- полином Лагранжа (многочлен минимальной степени, принимающий заданные значения в заданном наборе точек, то есть решающий задачу интерполяции): степень должна быть равна количеству точек (минус 1), но можно и больше, если че, т.е. $M >= N$
- почему? потому что полином Лагранжа - это просто сумма базисных функций вида $\prod_{j \neq i} (x - x_i) / (x_i - x_j)$, которые равны 1 при $i=j$, иначе 0, умноженных на $y_i$

3. **Рассмотрим** уравнение $y = X a$. При каком значении M существует единственное решение уравнения (и, соответственно, имеется единственное решение соответствующей оптимизационной задачи)?

$M = N$

4. **Рассмотрим** уравнение $y = X a$. Как мы уже убедились, решением этого уравнения будет также и решение задачи оптимизации $L(a) = \|Xa - y\|_2^2$ в том случае, если система совместна (имеет решение).

Однако, если матрица $X$ необратима (например, если она не квадратная), то, несмотря на то, что система уравнений может и не иметь решения, задача оптимизации в любом случае будет иметь решение. Найдите решение этой задачи оптимизации, исходя из условия $\forall i \ \frac{\partial L}{\partial a_i} = 0$. Считайте, что $M \leq N$

Это будет означать, что точного решения нет, но есть проекция, которая будут максимально близка к точному решению (метод наименьших квадратов):

$L(a) = (Xa-y)^{T}(Xa-y) \rightarrow \min _{a} \Leftrightarrow X^T X a = X^T y \Leftrightarrow a = (X^T X)^{-1}X^Ty$

5. **В случае**, когда $M > N$, решение задачи оптимизации не единственно. Существует целое векторное пространство, добавление любого элемента из которого к любому решению задачи минимизации приводит к другому решению той же задачи.

Найдите матрицу, собственные векторы которой, соответствующие ее нулевым собственным числам, образуют это векторное пространство (которое называется ядром или нулевым пространством матрицы $X$).

$Xa = \lambda a$ - собственные числа ($\lambda$) и векторы ($a$) матрицы $X$ по определению. Такие векторы, которые остаются коллинеарными себе в результате преобразований линейного оператора $X$

Но нас спрашивают про нулевые собственные числа, т.е. решения соответствующей однородной СЛАУ: $Xa=0$ 

И вообще, собственные векторы/числа имеют смысл только для квадратных матриц, поэтому $X^T X a = 0$, и ответ $X^T X$

\* Ортогональная матрица $Q: |Qx| = |x|$ (как оператор - не меняет длины векторов), а также $Q^T Q = E$. Это не в тему, просто

6.  **Как через все точки выборки** (всего N штук) может проходить бесконечное число полиномов степени M при M > N.

Так и в нейронных сетях: если данных мало, а параметров слишком много, то возможна ситуация, когда не единственный набор параметров решает задачу.

Это очень вредная ситуация, поскольку, если нейронная сеть очень хорошо приблизила тренировочные точки, cовсем не ясно, как она поведет себя на тестовых данных.

Более того, поскольку решений оптимизационной задачи в такой ситуации бесконечно много, то на тестовых данных она может работать бесконечно плохо.

Такая ситуация называется переобучение

Какова размерность нулевого пространства (ядра) из предыдущей задачи? ($M > N$)

**Ответ: $M-N$** (число )
Пусть A: X→Y — линейный оператор. (В нашем случае A: M→N)  
Ядром(нулевое пространство) линейного оператора A называется множество  KerA={x∈X∣Ax=0}  
Образом линейного оператора A называется множество  ImA={y∈Y∣y=Ax} (множество значений)  

Теорема (O ядре и базисе): dim(KerA)+dim(ImA)=n=dim(X)

7. **Задача**

Рассмотрим нейронную сеть, которая состоит из некоторого количества полносвязанных слоев с произвольной функцией активации.

Рассмотрим некоторый слой этой нейронной сети, в котором имеется $M + 1$ нейронов. Он принимает на вход некоторый вектор $\vec{x}$  размерности $N + 1$, который вычисляется предшествующими слоями, а возвращает вектор $\vec{y}$ размерности $M$, который затем обрабатывается последующими слоями. Этот можно записать следующим образом:

$\vec{y} = f \left(W^T \vec{x} + \vec{b} \right),$

где $f$ -- функция активации, матрица $W$ состоит из векторов весов для всех нейронов данного слоя: $W = \left[ \vec{w}_{0} , \vec{w}_{1}, \dots , \vec{w}_{M} \right]$, а вектор $\vec{b} = \left[ b_0, \dots, b_M \right]$ состоит из смещений для всех нейронов данного слоя.

Если расширить вектор $\vec{x}$ единицей (то есть, вместо вектора $\vec{x} = \left[ x_0, x_1, x_2, \dots , x_N \right]^T$, где $x_0, \dots , x_t$ - координаты вектора, в качестве входа в первый слой рассматривать вектор $\hat{\vec{x}} = \left[ 1, x_0, x_1, x_2, \dots , x_N \right]^T$, то первый слой нейронной сети можно переписать следующим образом:

$\vec{y} = f \left(\hat{W}^T \hat{\vec{x}}\right),$

где матрица $\hat{W} = \left[ \hat{w}_{0}, \hat{w}_{1}, \dots , \hat{w}_{M} \right]$ состоит из векторов параметров для ВСЕХ параметров всех нейронов первого слоя, включая смещения и все веса: $\hat{w}_{i} = \left[ b_i, \vec{w}_{i}^T \right]^T$ - вектор параметров нейрона данного слоя с номером $i$.

Если имеется несколько входных векторов в данный слой в обучающей выборке: $X = [\vec{x}_0, \vec{x}_1, \dots, \vec{x}_N]$, то мы можем вычислить результаты работы первого слоя нейронной сети сразу для всех векторов из обучающей выборки:

$Y = f \left( \hat{W}^T \hat{X} \right).$

**ЗАДАЧА**

Найдите через $\hat{W}, \hat{X}$ такую матрицу, собственные векторы которой, соответствующие её нулевым собственным значениям, задают пространство, добавление любого элемента которого к вектору параметров любого нейрона данного слоя, не приводит к изменению работы данного слоя (и, следовательно, всей сети) ни на одном из векторов обучающей выборки.

**РЕШЕНИЕ (херь какая-то но правильно вроде)**

Не меняет, значит: $f(\hat{W}^T \hat{X}) = f ([\hat{W}^T \oplus C] \hat{X}) \rightarrow C: CX = 0 \rightarrow C = X\cdot X^T $ (почему?)

Не меняет, значит производная по параметрам =0: $X = 0$. Но $X$ не квадратная, поэтому  $ X\cdot X^T $

**Допустим, у нас есть функция** $f(X) = \sum{log_{e}(x_{ij} + 1)}$, где $X$ - тензор размера 2x2. Мы выбрали начальное приближение $X^{t=0} = [[1, 2], [4, 5]]$. И шаг градиентного спуска $\alpha=10$.
Чему будет равен $X^{t=1}$? 

In [4]:
import torch

alpha = 10
x = torch.tensor(
    [[1.,  2.],
     [4.,  5.]], requires_grad=True)


function = (x+1).log().sum()
function.backward()

print (x, '<- X[t=0]')
print(x.grad, '<- gradient')
print(x - alpha * x.grad, '<- X[t=1]')

tensor([[1., 2.],
        [4., 5.]], requires_grad=True) <- X[t=0]
tensor([[0.5000, 0.3333],
        [0.2000, 0.1667]]) <- gradient
tensor([[-4.0000, -1.3333],
        [ 2.0000,  3.3333]], grad_fn=<SubBackward0>) <- X[t=1]


Реализуйте расчет градиента для функции $f(w) = \prod\limits_{i,j}{log_{e}(log_{e}({w_{i,j} + 7}}))$ в точке $w = [[5, 10], [1, 2]]$

In [24]:
w =  torch.tensor([[5, 10], [1, 2]]).float()
w.requires_grad = True

w =  torch.FloatTensor([[5,10],[1,2]]).requires_grad_(True) # как вариант

function = torch.log(torch.log(w + 7)).prod()
function = (w + 7).log().log().prod()   # ого, прикольно

function.backward()

print(w.grad)
function

tensor([[0.0201, 0.0109],
        [0.0449, 0.0351]])


tensor(0.5463, grad_fn=<ProdBackward0>)

Реализуйте градиентный спуск для той же функции $f(w) = \prod\limits_{i,j}{log_{e}(log_{e}({w_{ij} + 7}}))$

Пусть начальным приближением будет $w^{t=0} = [[5, 10], [1, 2]]$, шаг градиентного спуска $alpha=0.001$.

Чему будет равен $w^{t=500}$?

In [26]:
w = torch.tensor([[5., 10.], [1., 2.]], requires_grad=True)
alpha = 0.001

for _ in range(500):
    # critical: calculate the function inside the loop
    function = (w + 7).log().log().prod()
    function.backward()
    w.data -= alpha * w.grad # put code here
    w.grad.zero_()

print(w)

tensor([[4.9900, 9.9948],
        [0.9775, 1.9825]], requires_grad=True)


**Семинар** по градиентному спуску в `./Neural_Networks_and_CV`